In [1]:
require(readr)

Loading required package: readr


In [6]:
normalize <- function(x){(x-min(x))/(max(x)-min(x))}

In [87]:
get_score <- function(gene.vec, weight.df){
    scores <- c()
    for (gene in gene.vec){
        score <- 0
        cur.score <- 0
        for (i in 1:nrow(weight.df)){
            gene.group <- weight.df$genes[[i]]
            cur.score <- match(gene, gene.group)
            if (is.na(cur.score)){
                cur.score <- 0
            } else {
                cur.score <- (length(gene.group) - cur.score) * weight.df$weight[i]
            }
            score <- score + cur.score
            #print(score)
        }
        scores <- c(scores, score)
    }
    return(scores)
}

In [28]:
best.genes <- as.data.frame(read_tsv("../results/best_genes.tsv"))

Parsed with column specification:
cols(
  model = col_character(),
  genes = col_character(),
  auc = col_double()
)


In [29]:
best.genes

model,genes,auc
LR,"ERBB2,PIK3R1,ABL2,BAD,ABL1,MAPK9,SOS2,STAT5B,CBL,SRC,CBLC",0.7921931
rbfSVM,"ERBB2,SRC,PIK3R5,SHC4,ABL1,MAPK9,BTC,AKT1,CAMK2A,RPS6KB1,CAMK2G",0.7585523
RF,"ERBB2,PRKCB,MAPK9,MAP2K2,SHC2,CBLC,NRAS,PIK3CD,PIK3R2,JUN,HBEGF",0.7792454
GradientBoost,"ERBB2,SRC,PIK3CG,MAPK1,CAMK2G,AKT1,STAT5B,MAP2K4,ERBB4,PLCG1,STAT5A",0.7803439
LinearSVM,"ERBB2,RPS6KB1,ABL1,ABL2,AKT1,AKT3,BTC,CAMK2A,AKT2,ARAF,SHC3",0.7585523


In [30]:
best.genes$weight <- normalize(best.genes$auc) + 1

In [34]:
best.genes$genes <- strsplit(best.genes$genes, split = ',')

In [38]:
best.genes$genes[[1]]

[1] "ERBB2"  "PIK3R1" "ABL2"   "BAD"    "ABL1"   "MAPK9"  "SOS2"   "STAT5B"
 [9] "CBL"    "SRC"    "CBLC"

In [41]:
all.genes <- as.data.frame(read_tsv("../data/genes.txt", col_names = F))
names(all.genes) <- c("gene")

Parsed with column specification:
cols(
  X1 = col_character()
)


In [88]:
all.genes$score <- get_score(all.genes$gene, best.genes)

In [102]:
all.genes <- all.genes[order(all.genes$score, decreasing = T),]
row.names(all.genes) <- seq(length = nrow(all.genes))

#all.genes

top.genes <- all.genes[which(all.genes$score > 0),]

In [103]:
top.genes

gene,score
ERBB2,72.628893
MAPK9,27.920944
ABL1,26.000000
SRC,25.829942
ABL2,23.000000
PIK3R1,18.000000
AKT1,17.238856
PRKCB,14.536062
BAD,14.000000
PIK3CG,13.182170
